In [10]:
import os
import re
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import time

OSU_API_KEY = "839b8920c801ccdc4f56a01f533c941015e35d7f"  # <-- PUT YOUR API KEY HERE
STAGE_NAMES = [
    'Grand Finals', 'Finals', 'Semifinals', 'Quarterfinals',  "Play-off 1", "Play-off 2", "Playoffs",
    'Round of 16', 'Round of 32', 'Qualifiers'
]
POW = 30  # For 4K OSU

def extract_id(cell, kind="map"):
    if pd.isna(cell):
        return None
    s = str(cell).strip()
    if s.isdigit():
        return s
    try:
        if float(s).is_integer():
            return str(int(float(s)))
    except Exception:
        pass
    if kind == "map":
        m = re.search(r'#mania/(\d+)', s)
        if m:
            return m.group(1)
        m2 = re.search(r'/(\d+)$', s)
        if m2:
            return m2.group(1)
    elif kind == "match":
        m = re.search(r'/matches/(\d+)', s)
        if m:
            return m.group(1)
    return None

def load_match_links(stage, TOURNEY_DIR):
    path = os.path.join(TOURNEY_DIR, f"{stage}_matches.csv")
    if not os.path.exists(path):
        return []
    df = pd.read_csv(path)
    links = df.iloc[:,0].dropna().tolist()
    match_ids = []
    for cell in links:
        mid = extract_id(cell, kind="match")
        if mid:
            match_ids.append(mid)
    return match_ids

def get_match_scores(match_id):
    url = f"https://osu.ppy.sh/api/get_match?k={OSU_API_KEY}&mp={match_id}"
    try:
        resp = requests.get(url, timeout=20)
        data = resp.json()
        if not isinstance(data, dict):
            print(f"Non-dict response for match {match_id}: {data}")
            return []
        if "games" not in data:
            print(f"No 'games' in match {match_id} response: {data}")
            return []
        scores = []
        for g in data["games"]:
            map_id = g.get("beatmap_id", None)
            game_scores = g.get("scores", [])
            if not game_scores:
                print(f"Game {g.get('game_id')} in match {match_id} has no scores.")
            for s in game_scores:
                scores.append({
                    "user_id": s["user_id"],
                    "map_id": map_id,
                    "score": int(s["score"]),
                    "match_id": match_id
                })
        return scores
    except Exception as e:
        print(f"Failed to fetch match {match_id}: {e}")
    return []

def get_username(user_id, username_cache):
    if user_id in username_cache:
        return username_cache[user_id]
    url = f"https://osu.ppy.sh/api/get_user?k={OSU_API_KEY}&u={user_id}"
    try:
        resp = requests.get(url, timeout=10)
        data = resp.json()
        if isinstance(data, list) and data and "username" in data[0]:
            username = data[0]["username"]
        else:
            username = "Banned or Unavailable"
    except Exception as e:
        print(f"Failed to fetch username for {user_id}: {e}")
        username = "Banned or Unavailable"
    username_cache[user_id] = username
    return username

def process_tournament(TOURNEY_DIR):
    username_cache = {}
    print(f"\n\n========== Processing {TOURNEY_DIR} ==========")
    for stage in STAGE_NAMES:
        match_ids = load_match_links(stage, TOURNEY_DIR)
        if not match_ids:
            # print(f"No matches found for {stage}, skipping.")
            continue
        # print(match_ids)
        # print(f"Processing stage: {stage} ({len(match_ids)} matches)")

        all_scores = []
        for match_id in tqdm(match_ids):
            scores = get_match_scores(match_id)
            all_scores.extend(scores)
            # print(f"Fetched {len(scores)} scores for match {match_id}")
            # print(scores)

        if not all_scores:
            # print(f"No scores found for {stage}, skipping.")
            continue

        scores_df = pd.DataFrame(all_scores)
        scores_df = scores_df[scores_df["score"] > 0]
        if scores_df.empty:
            # print(f"No nonzero scores for {stage}, skipping.")
            continue

        mappool_path = os.path.join(TOURNEY_DIR, f"{stage}_maps.csv")
        if not os.path.exists(mappool_path):
            # print(f"No mappool file for {stage}, skipping.")
            continue
        mappool_df = pd.read_csv(mappool_path)

        mappool_ids = set()
        for cell in mappool_df.iloc[:,0].dropna():
            mapid = extract_id(cell, kind="map")
            if mapid:
                mappool_ids.add(mapid)

        # print(f"Extracted {len(mappool_ids)} mappool IDs from {stage}_maps.csv")

        if not mappool_ids:
            print(f"No beatmap IDs could be extracted from {stage}_maps.csv, skipping stage.")
            continue

        scores_df = scores_df[scores_df["map_id"].astype(str).isin(mappool_ids)]
        # print(scores_df)

        scores_df_path = os.path.join(TOURNEY_DIR, f"{stage}_scores.csv")
        scores_df.to_csv(scores_df_path, index=False)

        if scores_df.empty:
            # print(f"No mappool maps played for {stage}, skipping.")
            continue

        medians = scores_df.groupby("map_id")["score"].median()
        player_pscores = {}
        player_maps_played = {}
        player_total_possible_maps = {}

        # print(f"Calculated medians for {len(medians)} maps:")
        # print(medians)
        for player in scores_df["user_id"].unique():
            player_df = scores_df[scores_df["user_id"] == player]
            n_maps_played = len(player_df)
            n_unique_maps_played = player_df["map_id"].nunique()
            matches_played = player_df["match_id"].unique()
            maps_in_matches = scores_df[scores_df["match_id"].isin(matches_played)][["match_id", "map_id"]].drop_duplicates()["map_id"]
            n_possible_map_plays = len(maps_in_matches)
            player_maps_played[player] = n_maps_played
            player_total_possible_maps[player] = n_possible_map_plays

            ratio_terms = []
            for _, row in player_df.iterrows():
                s = row["score"]
                m = medians[row["map_id"]]
                ratio = (s / m) ** POW if m > 0 else 0
                ratio_terms.append(ratio)
            if not ratio_terms:
                continue
            part1 = np.mean(ratio_terms)

            N_js = []
            for match_id in matches_played:
                match_df = scores_df[scores_df["match_id"] == match_id]
                plays_per_player = match_df.groupby("user_id")["map_id"].count()
                mean_plays = plays_per_player.mean()
                N_js.append(mean_plays)
            if N_js:
                normalization = (n_maps_played / sum(N_js)) ** (1/3)
            else:
                normalization = 1

            pscore = part1 * normalization
            player_pscores[player] = pscore

        out_list = []
        for pid, score in player_pscores.items():
            name = get_username(pid, username_cache)
            maps_played = f"{player_maps_played[pid]}/{player_total_possible_maps[pid]}"
            out_list.append({
                "Player ID": pid,
                "Player Name": name,
                "pScore": score,
                "Maps Played": maps_played
            })
        out_df = pd.DataFrame(out_list)
        out_df = out_df.sort_values("pScore", ascending=False)
        out_path = os.path.join(TOURNEY_DIR, f"{stage}_leaderboard.csv")
        out_df.to_csv(out_path, index=False)
        print(f"Leaderboard saved to {out_path}")

if __name__ == "__main__":
    ROOT_DIR = os.getcwd()  # or specify a path, e.g. "osu_tournaments"
    for folder in os.listdir(ROOT_DIR):
        path = os.path.join(ROOT_DIR, folder)
        if not os.path.isdir(path):
            continue
        if folder == "manual_work":
            continue
        # Add further checks if your tournament folders have a pattern (e.g. folder name starts with a year, etc)
        process_tournament(path)
        print(f"Finished processing {folder}")
    print("All tournaments processed.")



========== Processing c:\Users\Acer\Documents\skillbans-in-2024-main\tournament-elo\.vscode ==========
Finished processing .vscode


========== Processing c:\Users\Acer\Documents\skillbans-in-2024-main\tournament-elo\4 Digit osumania World Cup 2024 ==========


100%|██████████| 2/2 [00:00<00:00,  3.60it/s]


Leaderboard saved to c:\Users\Acer\Documents\skillbans-in-2024-main\tournament-elo\4 Digit osumania World Cup 2024\Grand Finals_leaderboard.csv


100%|██████████| 4/4 [00:01<00:00,  3.67it/s]


Leaderboard saved to c:\Users\Acer\Documents\skillbans-in-2024-main\tournament-elo\4 Digit osumania World Cup 2024\Finals_leaderboard.csv


...

Game 611942404 in match 117265968 has no scores.


 93%|█████████▎| 14/15 [00:04<00:00,  3.53it/s]

Game 612042154 in match 117281560 has no scores.


100%|██████████| 15/15 [00:04<00:00,  3.38it/s]


Leaderboard saved to c:\Users\Acer\Documents\skillbans-in-2024-main\tournament-elo\Touhou Project Mania Cup 4th\Round of 32_leaderboard.csv
Finished processing Touhou Project Mania Cup 4th
All tournaments processed.


# need to remove ALL SVS so svse is ignored as well

# looks like this works... double checked with MWC 4k finals already and it seems like pscore is the same :D, can start extending this to all stages

# though some players can be extreme outliers....

takes 23 minutes to run this block

In [ ]:
import os
import pandas as pd
import numpy as np

# 1. Tournament folder -> tier mapping (fill this in with your real folder names and tiers)
TOURNAMENT_TIERS = {
    "4 Digit osumania World Cup 3": "A",
    "4 Digit osumania World Cup 4": "A",
    "4 Digit osumania World Cup 2024": "A",
    "osumania 4K World Cup 2021": "S",
    "osumania 4K World Cup 2022": "S",
    "osumania 4K World Cup 2023": "S",
    "osumania 4K World Cup 2024": "S",
    "osumania 4K Chinese National Cup 2022": "A",
    "osumania 4K Chinese National Cup 2023": "A",
    "osumania 4K Chinese National Cup 2024": "A",
    "osumania 4K Indonesia Cup 2022": "A",
    "osumania Malaysia Tournament 4": "A",
    "osumania LN Tournament 3": "S",
    "GB Cup 2023 Autumn": "A",
    "GB Cup 2024 Spring": "S",
    "JHC 2024": "A",
    "Meow Mania": "S",
    "Speed of Light 2 Lucha Libre Edition": "S",
    "Speed of Light 3": "S",
    "Springtime Osumania Free-for-all Tournament 5": "S",
    "Springtime Osumania Free-for-all Tournament 6": "S",
    "Touhou Project Mania Cup 3rd": "A",
    "Touhou Project Mania Cup 4th": "S",
}

def weighted_mean(group):
    return np.average(group["pScore"], weights=group["Weight"])

# 2. Tier to numerical weight mapping
TIER_WEIGHTS = {"S": 1.1, "A": 0.85, "B": 0.7, "C": 0.6}

def iqr_mean(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr_values = series[(series >= q1) & (series <= q3)]
    if not iqr_values.empty:
        return iqr_values.mean()
    else:
        return np.nan

ROOT_DIR = os.getcwd()  # Or set this to your tournaments root directory
all_pscores = []

for folder in os.listdir(ROOT_DIR):
    folder_path = os.path.join(ROOT_DIR, folder)
    if not os.path.isdir(folder_path) or folder == "manual_work":
        continue
    tier = TOURNAMENT_TIERS.get(folder, "C")
    weight = TIER_WEIGHTS.get(tier, 1.0)
    for file in os.listdir(folder_path):
        if file.endswith("_leaderboard.csv"):
            csv_path = os.path.join(folder_path, file)
            df = pd.read_csv(csv_path)
            required = {"Player ID", "Player Name", "pScore", "Maps Played"}
            if not required.issubset(df.columns):
                print(f"Skipping {csv_path} due to missing columns: {required - set(df.columns)}")
                continue
            df["Maps Played Num"] = df["Maps Played"].astype(str).str.extract(r"^(\d+)")
            df["Maps Played Num"] = pd.to_numeric(df["Maps Played Num"], errors="coerce").fillna(0).astype(int)
            df["Tier"] = tier
            df["Weight"] = weight
            # turn pscore to float and add weighted_pScore column
            df["weighted_pScore"] = df["pScore"] * df["Weight"]
            all_pscores.append(df[["Player ID", "Player Name", "pScore", "weighted_pScore", "Maps Played Num", "Tier", "Weight"]])
            # #drop columns that are not needed for the final summary
            # all_pscores = all_pscores.drop(columns=["pscore", "Weight"])

if all_pscores:
    combined = pd.concat(all_pscores, ignore_index=True)
    display(combined)
    # Weighted mean pScore

    summary = (
        combined.groupby(["Player ID", "Player Name"], as_index=False)
        .agg(
            median_pScore=("weighted_pScore", "median"),
            iqr_mean_pScore=("weighted_pScore", iqr_mean),
            total_maps_played=("Maps Played Num", "sum"),
            num_appearances=("pScore", "count")
        )
        .sort_values("median_pScore", ascending=False)
    )

    summary = summary[summary["num_appearances"] > 7] # roughly 3-4 tournaments played, use number of tournament appreances as a filter instead TODO
    summary.to_csv("combined_player_weighted_stats.csv", index=False)
    print("Combined leaderboard with tournament weighting saved as combined_player_weighted_stats.csv")
else:
    print("No leaderboard files found.")

,Player ID,Player Name,pScore,weighted_pScore,Maps Played Num,Tier,Weight
0,14892447,JayLye,1.497120e+00,1.272552e+00,10,A,0.85
1,14177626,charlie72,1.488531e+00,1.265251e+00,16,A,0.85
2,15179858,Tonels,1.391576e+00,1.182839e+00,15,A,0.85
3,27540621,angela202133,1.359491e+00,1.155567e+00,15,A,0.85
4,26233321,[LS]Vixile,1.326240e+00,1.127304e+00,9,A,0.85
...,...,...,...,...,...,...,...
6271,11799103,yoppy-yugen,5.679357e-01,6.247293e-01,4,S,1.10
6272,32901596,MSMS33,5.144673e-01,5.659140e-01,3,S,1.10
6273,16191180,Shice2566,4.943838e-01,5.438222e-01,2,S,1.10
6274,17753122,invadey,3.644913e-01,4.009404e-01,1,S,1.10


Combined leaderboard with tournament weighting saved as combined_player_weighted_stats.csv


In [29]:
import os
import pandas as pd

def find_player_pscores(player_id, root_dir=None, save_csv=True):
    if root_dir is None:
        root_dir = os.getcwd()
    player_rows = []

    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        if not os.path.isdir(folder_path) or folder == "manual_work":
            continue
        for file in os.listdir(folder_path):
            if file.endswith("_leaderboard.csv"):
                csv_path = os.path.join(folder_path, file)
                try:
                    df = pd.read_csv(csv_path)
                except Exception as e:
                    print(f"Error reading {csv_path}: {e}")
                    continue
                if "Player ID" not in df.columns:
                    continue
                # Find player by ID (may need to str for match)
                rows = df[df["Player ID"].astype(str) == str(player_id)].copy()
                if not rows.empty:
                    # Extract stage name from file name (e.g. "Finals_leaderboard.csv" -> "Finals")
                    stage = file.replace("_leaderboard.csv", "")
                    rows["Tournament"] = folder
                    rows["Stage"] = stage
                    player_rows.append(rows)
    
    if not player_rows:
        print(f"No results found for player ID {player_id}.")
        return pd.DataFrame()
    
    result = pd.concat(player_rows, ignore_index=True)
    # Reorder for clarity
    cols = ["Tournament", "Stage", "Player ID", "Player Name", "pScore", "Maps Played"]
    result = result[[col for col in cols if col in result.columns]]
    # Optionally save as CSV
    if save_csv:
        out_csv = f"pscore_search_{player_id}.csv"
        result.to_csv(out_csv, index=False)
        print(f"Results saved to {out_csv}")
    return result

# Example usage:
result_df = find_player_pscores(player_id=13385865)


Results saved to pscore_search_13385865.csv
